In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF


In [2]:
# Load the data from the corrected Google Drive link
url = 'https://drive.google.com/uc?export=download&id=17oOO_s6BbyVPzhaM-5EzKLrjoe3_ft90'
data = pd.read_csv(url)
print(data.head())  # Display the first few rows to ensure data is loaded correctly

      Order # Fulfillment Date and Time Stamp Currency    Subtotal  \
0  R121113121                             NaN      INR  ₹ 2,299.00   
1  R472890631                             NaN      INR  ₹ 2,299.00   
2  R004476488                             NaN      INR  ₹ 2,299.00   
3  R526038353                             NaN      INR  ₹ 2,299.00   
4  R658530771                             NaN      INR    ₹ 349.00   

  Shipping Method Shipping Cost Tax Method   Taxes       Total Coupon Code  \
0      Ships Free        ₹ 0.00        NaN  ₹ 0.00  ₹ 2,299.00         NaN   
1      Ships Free        ₹ 0.00        NaN  ₹ 0.00  ₹ 2,299.00         NaN   
2      Ships Free        ₹ 0.00        NaN  ₹ 0.00  ₹ 2,299.00         NaN   
3      Ships Free        ₹ 0.00        NaN  ₹ 0.00  ₹ 2,299.00         NaN   
4      Ships Free        ₹ 0.00        NaN  ₹ 0.00    ₹ 349.00         NaN   

   ... Tracking # Special Instructions  \
0  ...        NaN                  NaN   
1  ...        NaN         

In [3]:
print(data.columns)

Index(['Order #', 'Fulfillment Date and Time Stamp', 'Currency', 'Subtotal',
       'Shipping Method', 'Shipping Cost', 'Tax Method', 'Taxes', 'Total',
       'Coupon Code', 'Coupon Code Name', 'Discount', 'Billing Name',
       'Billing Country', 'Billing Street Address', 'Billing Street Address 2',
       'Billing City', 'Billing State', 'Billing Zip', 'Shipping Name',
       'Shipping Country', 'Shipping Street Address',
       'Shipping Street Address 2', 'Shipping City', 'Shipping State',
       'Shipping Zip', 'Gift Cards', 'Payment Method', 'Tracking #',
       'Special Instructions', 'LineItem Name', 'LineItem SKU',
       'LineItem Options', 'LineItem Add-ons', 'LineItem Qty',
       'LineItem Sale Price', 'Download Status', 'LineItem Type'],
      dtype='object')


# The Shipping Address Differs from the Billing Address

In [4]:
# Filter orders where the shipping address is different from the billing address
shipping_vs_billing = data[
    (data['Billing Street Address'] != data['Shipping Street Address']) |
    (data['Billing City'] != data['Shipping City']) |
    (data['Billing State'] != data['Shipping State']) |
    (data['Billing Zip'] != data['Shipping Zip'])
]

# Save to CSV
shipping_vs_billing.to_csv('shipping_vs_billing.csv', index=False)


# Multiple Orders of the Same Item

In [5]:
# Group by Order and LineItem SKU to find multiple orders of the same item
multiple_orders_same_item = data.groupby(['Order #', 'LineItem SKU']).filter(lambda x: len(x) > 1)

# Save to CSV
multiple_orders_same_item.to_csv('multiple_orders_same_item.csv', index=False)


# Unusually Large Orders

In [6]:
# Clean the 'Total' column and filter orders exceeding the threshold (e.g., ₹ 10,000)
data['Total_Clean'] = pd.to_numeric(data['Total'].replace('[₹,]', '', regex=True), errors='coerce')
unusually_large_orders = data[data['Total_Clean'] > 10000]

# Save to CSV
unusually_large_orders.to_csv('unusually_large_orders.csv', index=False)


# Multiple Orders to the Same Address with Different Payment Methods

In [7]:
# Filter orders by grouping by address and checking for different payment methods
multiple_payment_methods = data.groupby('Shipping Street Address').filter(
    lambda x: x['Payment Method'].nunique() > 1
)

# Save to CSV
multiple_payment_methods.to_csv('multiple_payment_methods.csv', index=False)


# Unexpected International Orders

In [8]:
# Filter orders where the country is not India (adjust based on your expected regions)
unexpected_international_orders = data[
    (data['Billing Country'] != 'India') | (data['Shipping Country'] != 'India')
]

# Save to CSV
unexpected_international_orders.to_csv('unexpected_international_orders.csv', index=False)


In [9]:
# Create PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)


In [10]:
# Add details for each section
pdf.cell(200, 10, txt="Fake Buyer Identification Report", ln=True, align="C")


In [11]:
# Add each section with results
pdf.cell(200, 10, txt="1. Orders with Different Shipping and Billing Addresses", ln=True)
pdf.cell(200, 10, txt=f"Total: {len(shipping_vs_billing)}", ln=True)


In [12]:
pdf.cell(200, 10, txt="2. Multiple Orders of the Same Item", ln=True)
pdf.cell(200, 10, txt=f"Total: {len(multiple_orders_same_item)}", ln=True)

In [13]:
pdf.cell(200, 10, txt="3. Unusually Large Orders", ln=True)
pdf.cell(200, 10, txt=f"Total: {len(unusually_large_orders)}", ln=True)


In [14]:
pdf.cell(200, 10, txt="4. Multiple Orders to Same Address with Different Payment Methods", ln=True)
pdf.cell(200, 10, txt=f"Total: {len(multiple_payment_methods)}", ln=True)

In [15]:
pdf.cell(200, 10, txt="5. Unexpected International Orders", ln=True)
pdf.cell(200, 10, txt=f"Total: {len(unexpected_international_orders)}", ln=True)


In [16]:
# Save PDF
pdf.output("Fake_Buyer_Identification_Report.pdf")

''